In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import itertools
import numpy as np

In [5]:
#scraper

tours = ['tour-de-france', 'giro-d-italia', 'vuelta-a-espana']
#years = [2020, 2019, 2018, 2017, 2016, 2015, 2014]
years = [2022]#, 2021, 2013, 2012, 2011, 2010]
years = [2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009]

In [5]:
tours = ['vuelta-a-espana']
years = [2022]

In [6]:
def scrape_participants(tour, year):
    # define url for startlist
    
    url = f'https://www.procyclingstats.com/race/{tour}/{year}/stage-21/startlist'
    print(url)
    
    #scrape page
    response = requests.get(url).content
    soup = BeautifulSoup(response)
    
    #get all_teams
    all_teams = soup.find_all('li', class_='team')
    
    #loop over teams
    master_ls = []
    for t in all_teams:
        team = t.a.text
        riders = t.find_all('a', class_='blue')
        for r in riders:
            dict = {}
            rider = r.span.text
            href = r['href']
            dict['rider'] = href.split('/')[-1]
            dict['team'] = team
            dict['href'] = href
            dict['tour'] = tour
            dict['year'] = int(year)
            master_ls.append(dict)
            
    return master_ls

In [7]:
participants_ls = []

for y, t in list(itertools.product(years, tours)):
    participants_ls.append(scrape_participants(t, y))

https://www.procyclingstats.com/race/vuelta-a-espana/2022/stage-21/startlist


In [8]:
pd.DataFrame(list(itertools.chain(*participants_ls))).to_csv('new_data/participants_vuelta2022.csv')

In [9]:
participants_df = pd.DataFrame(list(itertools.chain(*participants_ls)))

In [10]:
from matplotlib.pyplot import text


def scrape_performance(rider, endpoint, year):
    
    #set up
    base_url = 'https://www.procyclingstats.com/'
    url = base_url+endpoint+'/'+str(year)
    
    response = requests.get(url).content
    soup = BeautifulSoup(response)
    
    result_ls = []
    
    #get stage_race results
    stage_races = soup.find_all('tr', {'data-main': '0'})
    
    for o in stage_races:
        dict = {}
        o = o.find_all('td')
        dict['name'] = rider
        dict['year'] = str(year)
        dict['type'] = 'etappe'
        dict['date'] = o[0].text
        if len(dict['date']) == 0:
            dict['type'] = 'gc'
        dict['result'] = o[1].text
        dict['gc'] = o[2].text
        try:
            dict['icon'] = o[3].find('span', class_='icon')['class'][-1]
        except TypeError:
            dict['icon'] = 'stage'
        dict['race_ref'] = o[4].a['href']
        dict['race_name'] = dict['race_ref'].split('/')[1]
        dict['race_detail'] = o[4].a.text
        try:
            dict['race_rank'] = o[4].a.span.text
        except AttributeError:
            dict['race_rank'] = o[4].a.span
        dict['distance'] = o[5].text
        result_ls.append(dict)
    
    #get one day race results
    one_day_races = soup.find_all('tr', {'data-main': '1'})
    
    for o in one_day_races:
        dict = {}
        o = o.find_all('td')
        dict['name'] = rider
        dict['year'] = str(year)
        dict['type'] = 'one_day'
        dict['date'] = o[0].text
        dict['result'] = o[1].text
        dict['gc'] = o[2].text
        try:
            dict['icon'] = o[3].find('span', class_='icon')['class'][-1]
        except TypeError:
            dict['icon'] = 'stage'
        dict['race_ref'] = o[4].a['href']
        dict['race_name'] = dict['race_ref'].split('/')[1]
        dict['race_detail'] = o[4].a.text
        try:
            dict['race_rank'] = o[4].a.span.text
        except AttributeError:
            dict['race_rank'] = o[4].a.span
        dict['distance'] = o[5].text
        result_ls.append(dict)
    
    return result_ls

In [11]:
performance_ls = []

for index, row in participants_df.iterrows():
    performance_ls.append(scrape_performance(row['rider'], row['href'], row['year']))

In [12]:
performance_df = pd.DataFrame(list(itertools.chain(*performance_ls)))

In [13]:
performance_df.to_csv('new_data/raw_performance_vuelta2022.csv')

In [14]:
stage_s = list(np.arange(2,32,2))+list(np.arange(32,48,4))+[50]
stage_s_i = list(np.arange(1,21,1))
stage_s_dict = dict(zip(stage_s_i, stage_s[::-1]))

def clean_df(ls):
    df = pd.DataFrame(ls)
    
    index_drop = df[df['result']==''].index

    dropped_df = df.drop(index_drop)

    index_drop = dropped_df[dropped_df['type']=='gc'].index

    dropped_df = dropped_df.drop(index_drop)
    
    dropped_df['date'] = pd.to_datetime(dropped_df['date'] + '.' + dropped_df['year'], infer_datetime_format=True)
    
    dropped_df['result'] =  dropped_df['result'].replace('DNF', 0).replace('DNS', 0).replace('OTL', 0).replace('DSQ', 0).replace('DF', 0).astype('int')
    
    dropped_df['points'] = dropped_df['result'].map(stage_s_dict).fillna('0').astype('int')
    
    #depreciated -> for gc 
    #stages_df = df.loc[index_drop][['race_name', 'race_rank']]#.to_dict(orient='records')
    #stages_df = stages_df.set_index('race_name').to_dict()['race_rank']
    
    return dropped_df


In [15]:
performance_clean = clean_df(list(itertools.chain(*performance_ls)))
performance_clean.to_csv('new_data/performance_clean_vuelta2022.csv')

In [16]:
performance_clean.race_ref.unique()

array(['race/tour-de-france/2022/stage-15',
       'race/tour-de-france/2022/stage-14',
       'race/tour-de-france/2022/stage-13',
       'race/tour-de-france/2022/stage-12',
       'race/tour-de-france/2022/stage-11',
       'race/tour-de-france/2022/stage-10',
       'race/tour-de-france/2022/stage-9',
       'race/tour-de-france/2022/stage-8',
       'race/tour-de-france/2022/stage-7',
       'race/tour-de-france/2022/stage-6',
       'race/tour-de-france/2022/stage-5',
       'race/tour-de-france/2022/stage-4',
       'race/tour-de-france/2022/stage-3',
       'race/tour-de-france/2022/stage-2',
       'race/tour-de-france/2022/stage-1', 'race/dauphine/2022/stage-8',
       'race/dauphine/2022/stage-7', 'race/dauphine/2022/stage-6',
       'race/dauphine/2022/stage-5', 'race/dauphine/2022/stage-4',
       'race/dauphine/2022/stage-3', 'race/dauphine/2022/stage-2',
       'race/dauphine/2022/stage-1',
       'race/itzulia-basque-country/2022/stage-6',
       'race/itzulia-basque-co

In [17]:

def get_profile(list):
    extra_info_ls = []
    i=0
    
    for ref in list:
        print(i/len(performance_clean.race_ref.unique()))
        #create url
        base_url = 'https://www.procyclingstats.com/'
        url = base_url + ref
        response = requests.get(url).content
        soup = BeautifulSoup(response)
        
        print(url)
        
        #get al info
        dict = {}
        try:
            stage = soup.find('ul', class_='infolist').find_all('li')
                
            dict['href'] = ref
            #get speed
            try:
                dict[stage[2].find_all('div')[0].text] = float(stage[2].find_all('div')[1].text.strip(' km/h'))
            except ValueError:
                dict[stage[2].find_all('div')[0].text] = np.nan
            #get distance
            try:
                dict[stage[4].find_all('div')[0].text.strip()] = float(stage[4].find_all('div')[1].text.strip(' km'))
            except ValueError:
                dict[stage[4].find_all('div')[0].text.strip()] = np.nan
            #get parcours type
            try:
                dict[stage[6].find_all('div')[0].text.strip()] = stage[6].find_all('div')[1].span['class'][-1]
            except (ValueError, TypeError):
                dict[stage[6].find_all('div')[0].text.strip()] = np.nan
            #get profile score
            try:
                dict[stage[7].find_all('div')[0].text.strip()] = int(stage[7].find_all('div')[1].text)
            except ValueError:
                dict['ProfileScore:'] = np.nan
            #get vert meters
            try:
                dict[stage[8].find_all('div')[0].text.strip()] = int(stage[8].find_all('div')[1].text)
            except (ValueError, IndexError):
                dict['Vert. meters:'] = np.nan
            #get startlist
            try:
                dict[stage[12].find_all('div')[0].text.strip()] = int(stage[12].find_all('div')[1].text)
            except (ValueError, IndexError):
                dict['Startlist quality score:'] = np.nan
            #get won how
            try:
                dict[stage[13].find_all('div')[0].text]= stage[13].find_all('div')[1].text
            except (ValueError, IndexError):
                dict['Won how:'] = np.nan
            
            extra_info_ls.append(dict)
            
            i += 1
        
        except AttributeError:
            print(ref)
            
    return extra_info_ls

In [28]:
new_ls = []

for i in range(1,22):
    new_ls.append(f'race/vuelta-a-espana/2022/stage-{i}')

In [30]:
vuelta_ls = get_profile(new_ls)

0.0
https://www.procyclingstats.com/race/vuelta-a-espana/2022/stage-1
0.0023752969121140144
https://www.procyclingstats.com/race/vuelta-a-espana/2022/stage-2
0.004750593824228029
https://www.procyclingstats.com/race/vuelta-a-espana/2022/stage-3
0.007125890736342043
https://www.procyclingstats.com/race/vuelta-a-espana/2022/stage-4
0.009501187648456057
https://www.procyclingstats.com/race/vuelta-a-espana/2022/stage-5
0.011876484560570071
https://www.procyclingstats.com/race/vuelta-a-espana/2022/stage-6
0.014251781472684086
https://www.procyclingstats.com/race/vuelta-a-espana/2022/stage-7
0.0166270783847981
https://www.procyclingstats.com/race/vuelta-a-espana/2022/stage-8
0.019002375296912115
https://www.procyclingstats.com/race/vuelta-a-espana/2022/stage-9
0.021377672209026127
https://www.procyclingstats.com/race/vuelta-a-espana/2022/stage-10
0.023752969121140142
https://www.procyclingstats.com/race/vuelta-a-espana/2022/stage-11
0.026128266033254157
https://www.procyclingstats.com/race/v

In [34]:
pd.DataFrame(vuelta_ls).to_csv('new_data/stages_only_vuelta22.csv')

In [31]:
extra_info_ls = get_profile(performance_clean.race_ref.unique())

0.0
https://www.procyclingstats.com/race/tour-de-france/2022/stage-15
0.0023752969121140144
https://www.procyclingstats.com/race/tour-de-france/2022/stage-14
0.004750593824228029
https://www.procyclingstats.com/race/tour-de-france/2022/stage-13
0.007125890736342043
https://www.procyclingstats.com/race/tour-de-france/2022/stage-12
0.009501187648456057
https://www.procyclingstats.com/race/tour-de-france/2022/stage-11
0.011876484560570071


KeyboardInterrupt: 

In [19]:
stages_df = pd.DataFrame(extra_info_ls).rename(columns={'href':'race_ref'})
stages_df.to_csv('new_data/stages_vuelta2022.csv')

In [20]:
performance_df.drop_duplicates(inplace=True)

In [21]:
stages_df

,race_ref,Avg. speed winner:,Distance:,Parcours type:,ProfileScore:,Vert. meters:,Startlist quality score:,Won how:,Race category:,Points scale:,Arrival:,Won how:
0,race/tour-de-france/2022/stage-15,45.429,202.5,p2,64.0,2416.0,1550.0,Sprint of large group,NaN,NaN,NaN,NaN
1,race/tour-de-france/2022/stage-14,42.638,192.5,p2,170.0,3441.0,1550.0,1.9 km solo,NaN,NaN,NaN,NaN
2,race/tour-de-france/2022/stage-13,45.667,192.6,p2,78.0,2109.0,1550.0,Sprint of small group,NaN,NaN,NaN,NaN
3,race/tour-de-france/2022/stage-12,33.534,165.1,p5,389.0,4660.0,1550.0,11 km solo,NaN,NaN,NaN,NaN
4,race/tour-de-france/2022/stage-11,35.275,151.7,p5,400.0,4070.0,1550.0,3.9 km solo,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
416,race/le-tour-de-bretagne/2022/stage-5,42.783,193.2,p3,63.0,2368.0,37.0,Sprint of large group,NaN,NaN,NaN,NaN
417,race/le-tour-de-bretagne/2022/stage-4,43.115,164.7,p3,70.0,2000.0,37.0,Sprint of large group,NaN,NaN,NaN,NaN
418,race/le-tour-de-bretagne/2022/stage-3,42.141,171.7,p3,62.0,2005.0,37.0,5.2 km solo,NaN,NaN,NaN,NaN
419,race/le-tour-de-bretagne/2022/stage-2,43.380,152.0,p2,55.0,1789.0,37.0,1 km solo,NaN,NaN,NaN,NaN


In [22]:
merged = performance_clean.merge(stages_df, on='race_ref')
merged['points'] = merged['points'].astype('float')
merged['adjusted_points'] = merged['points'] * merged['ProfileScore:']  * merged['Startlist quality score:']
merged.head()

,name,year,type,date,result,gc,icon,race_ref,race_name,race_detail,...,Parcours type:,ProfileScore:,Vert. meters:,Startlist quality score:,Won how:,Race category:,Points scale:,Arrival:,Won how:,adjusted_points
0,primoz-roglic,2022,etappe,2022-07-17,0,,stage,race/tour-de-france/2022/stage-15,tour-de-france,Stage 15 - Rodez › Carcassonne,...,p2,64.0,2416.0,1550.0,Sprint of large group,NaN,NaN,NaN,NaN,0.0
1,sepp-kuss,2022,etappe,2022-07-17,117,21,stage,race/tour-de-france/2022/stage-15,tour-de-france,Stage 15 - Rodez › Carcassonne,...,p2,64.0,2416.0,1550.0,Sprint of large group,NaN,NaN,NaN,NaN,0.0
2,danny-van-poppel,2022,etappe,2022-07-17,5,124,stage,race/tour-de-france/2022/stage-15,tour-de-france,Stage 15 - Rodez › Carcassonne,...,p2,64.0,2416.0,1550.0,Sprint of large group,NaN,NaN,NaN,NaN,3174400.0
3,bob-jungels,2022,etappe,2022-07-17,38,14,stage,race/tour-de-france/2022/stage-15,tour-de-france,Stage 15 - Rodez › Carcassonne,...,p2,64.0,2416.0,1550.0,Sprint of large group,NaN,NaN,NaN,NaN,0.0
4,christopher-froome,2022,etappe,2022-07-17,40,29,stage,race/tour-de-france/2022/stage-15,tour-de-france,Stage 15 - Rodez › Carcassonne,...,p2,64.0,2416.0,1550.0,Sprint of large group,NaN,NaN,NaN,NaN,0.0


In [23]:
merged['ProfileScore:'].isna()

0       False
1       False
2       False
3       False
4       False
        ...  
8400    False
8401    False
8402    False
8403    False
8404    False
Name: ProfileScore:, Length: 8405, dtype: bool

In [24]:
merged.to_csv('new_data/merged_clean_vuelta2022.csv')

In [25]:
merged['adjusted_points'] = merged['points'] * merged['ProfileScore:']  * merged['Startlist quality score:']
merged.dropna(subset='ProfileScore:',inplace=True)
merged.drop_duplicates(inplace=True)
merged = merged[merged['adjusted_points'] != 0]
merged.sort_values(by='adjusted_points').tail(50)

,name,year,type,date,result,gc,icon,race_ref,race_name,race_detail,...,Parcours type:,ProfileScore:,Vert. meters:,Startlist quality score:,Won how:,Race category:,Points scale:,Arrival:,Won how:,adjusted_points
1829,thymen-arensman,2022,etappe,2022-05-28,5,18,stage,race/giro-d-italia/2022/stage-20,giro-d-italia,Stage 20 - Belluno › Marmolada (Passo Fedaia),...,p5,392.0,4718.0,816.0,53.7 km solo,NaN,NaN,NaN,NaN,10235904.0
6026,clement-champoussin,2022,etappe,2022-06-17,2,25,stage,race/tour-de-suisse/2022/stage-6,tour-de-suisse,Stage 6 - Locarno › Moosalp,...,p5,304.0,4155.0,767.0,Sprint of small group,NaN,NaN,NaN,NaN,10259392.0
1134,brandon-mcnulty,2022,etappe,2022-03-12,6,18,stage,race/paris-nice/2022/stage-7,paris-nice,Stage 7 - Nice › Col de Turini,...,p5,303.0,3582.0,1158.0,Sprint of small group,NaN,NaN,NaN,NaN,10526220.0
37,thibaut-pinot,2022,etappe,2022-07-16,3,19,stage,race/tour-de-france/2022/stage-14,tour-de-france,Stage 14 - Saint-Etienne › Mende,...,p2,170.0,3441.0,1550.0,1.9 km solo,NaN,NaN,NaN,NaN,10540000.0
423,johan-esteban-chaves,2022,etappe,2022-06-12,4,,stage,race/dauphine/2022/stage-8,dauphine,Stage 8 - Saint-Alban-Leysse › Plateau de Sala...,...,p5,375.0,3782.0,788.0,Other,NaN,NaN,NaN,NaN,10638000.0
6020,aleksey-lutsenko,2022,etappe,2022-06-18,3,19,stage,race/tour-de-suisse/2022/stage-7,tour-de-suisse,Stage 7 - Ambri › Malbun,...,p5,347.0,3881.0,767.0,2 km solo,NaN,NaN,NaN,NaN,10645960.0
1106,simon-yates,2022,etappe,2022-03-13,1,,stage,race/paris-nice/2022/stage-8,paris-nice,Stage 8 - Nice › Nice,...,p4,184.0,2436.0,1158.0,19.2 km solo,NaN,NaN,NaN,NaN,10653600.0
430,primoz-roglic,2022,etappe,2022-06-11,2,1,stage,race/dauphine/2022/stage-7,dauphine,Stage 7 - Saint-Chaffrey › Vaujany,...,p5,311.0,3828.0,788.0,5.7 km solo,NaN,NaN,NaN,NaN,10782992.0
2041,santiago-buitrago-sanchez,2022,etappe,2022-05-22,2,17,stage,race/giro-d-italia/2022/stage-15,giro-d-italia,Stage 15 - Rivarolo Canavese › Cogne,...,p5,301.0,4019.0,816.0,18.8 km solo,NaN,NaN,NaN,NaN,10807104.0
1202,brandon-mcnulty,2022,etappe,2022-03-10,1,23,stage,race/paris-nice/2022/stage-5,paris-nice,Stage 5 - Saint-Just-Saint-Rambert › Saint-Sau...,...,p4,188.0,3408.0,1158.0,38.7 km solo,NaN,NaN,NaN,NaN,10885200.0


In [26]:
merged.groupby(['race_ref']).mean().sort_values(by='adjusted_points').tail(50)

,result,points,Avg. speed winner:,Distance:,ProfileScore:,Vert. meters:,Startlist quality score:,Race category:,Points scale:,Arrival:,Won how:,adjusted_points
race_ref,,,,,,,,,,,,
race/volta-ao-algarve/2022/stage-2,9.555556,24.222222,37.628,182.40,197.0,3183.0,502.0,NaN,NaN,NaN,NaN,2.395432e+06
race/faun-ardeche-classic/2022/result,9.571429,24.857143,37.944,168.50,216.0,3161.0,453.0,NaN,NaN,NaN,NaN,2.432222e+06
race/tirreno-adriatico/2022/stage-2,6.250000,33.000000,40.383,219.00,70.0,2144.0,1061.0,NaN,NaN,NaN,NaN,2.450910e+06
race/giro-d-italia/2022/stage-8,10.000000,24.500000,43.122,153.00,123.0,2243.0,816.0,NaN,NaN,NaN,NaN,2.459016e+06
race/tirreno-adriatico/2022/stage-4,12.000000,18.153846,41.989,202.00,131.0,3092.0,1061.0,NaN,NaN,NaN,NaN,2.523221e+06
race/vuelta-a-la-comunidad-valenciana/2022/stage-1,9.000000,25.600000,38.989,166.70,162.0,3164.0,610.0,NaN,NaN,NaN,NaN,2.529792e+06
race/itzulia-basque-country/2022/stage-4,10.333333,22.222222,43.605,185.60,154.0,2703.0,742.0,NaN,NaN,NaN,NaN,2.539289e+06
race/tour-de-suisse/2022/stage-1,9.125000,25.000000,41.487,177.60,135.0,2746.0,767.0,NaN,NaN,NaN,NaN,2.588625e+06
race/volta-ao-algarve/2022/stage-5,8.875000,26.000000,40.725,173.00,199.0,3056.0,502.0,NaN,NaN,NaN,NaN,2.597348e+06


In [133]:
sum(merged['ProfileScore:'].isna())

75274

In [282]:
diction = {}

stage = soup.find('ul', class_='infolist').find_all('li')
stage

[<li><div>Date:</div> <div>18 July 2021</div></li>,
 <li><div>Start time:</div> <div>16:30 </div></li>,
 <li><div>Avg. speed winner:</div> <div>40.748 km/h</div></li>,
 <li><div>Race category:</div> <div>ME - Men Elite</div></li>,
 <li><div>Distance: </div> <div>108.4 km</div></li>,
 <li><div>Points scale:</div> <div><a href="info.php?s=point-scales&amp;season=2021&amp;category=1&amp;scale=7">GT.A.Stage</a></div></li>,
 <li><div>Parcours type: </div> <div><span class="icon profile p1"></span></div></li>,
 <li><div>ProfileScore: </div> <div>14</div></li>,
 <li><div>Vert. meters:</div> <div>697</div></li>,
 <li><div>Departure:</div> <div><a href="location/chatou">Chatou</a></div></li>,
 <li><div>Arrival:</div> <div><a href="location/paris">Paris Champs-Élysées</a></div></li>,
 <li><div>Race ranking:</div> <div>1</div></li>,
 <li><div>Startlist quality score:</div> <div><a href="race/tour-de-france/2021/stage-21/startlist/lineup-quality">1646</a></div></li>,
 <li><div>Won how: </div> <div

In [292]:
diction['href'] = endpoint
#get speed
diction[stage[2].find_all('div')[0].text] = float(stage[2].find_all('div')[1].text.strip(' km/h'))
#get distance
diction[stage[4].find_all('div')[0].text.strip()] = float(stage[4].find_all('div')[1].text.strip(' km'))
#get parcours type
diction[stage[6].find_all('div')[0].text.strip()] = stage[6].find_all('div')[1].span['class'][-1]
#get profile score
diction[stage[7].find_all('div')[0].text.strip()] = int(stage[7].find_all('div')[1].text)
#get vert meters
diction[stage[8].find_all('div')[0].text.strip()] = int(stage[8].find_all('div')[1].text)
#get vert meters
diction[stage[12].find_all('div')[0].text.strip()] = int(stage[12].find_all('div')[1].text)
#get won how
diction[stage[13].find_all('div')[0].text]= stage[13].find_all('div')[1].text
diction

{'href': 'race/tour-de-france/2021/stage-21',
 'Avg. speed winner:': 40.748,
 'Distance: ': 108.4,
 'Parcours type:': 'p1',
 'ProfileScore:': 14,
 'Distance:': 108.4,
 'Vert. meters:': 697,
 'Startlist quality score:': 1646,
 'Won how: ': 'Sprint of large group'}

In [294]:
pd.DataFrame(dict)

AttributeError: 'dict' object has no attribute 'to_records'

In [259]:
stage[2].find_all('div')[0].text, stage[2].find_all('div')[1].text

('Avg. speed winner:', '40.748 km/h')

In [232]:
stages_df.set_index('race_name').to_dict()['race_rank']

{'tour-de-france': '(2.UWT)',
 'tour-of-slovenia': '(2.Pro)',
 'itzulia-basque-country': '(2.UWT)',
 'tirreno-adriatico': '(2.UWT)',
 'uae-tour': '(2.UWT)'}

In [187]:
tadej_df.drop(index_drop)

,year,type,date,result,gc,icon,race_ref,race_name,race_detail,race_rank,distance
0,2021,gc,,1,,st6,race/tour-de-france/2021/stage-21-youth,tour-de-france,Youth classification,None,
1,2021,gc,,1,,st7,race/tour-de-france/2021/stage-21-kom,tour-de-france,Mountains classification,None,
2,2021,gc,,8,,st5,race/tour-de-france/2021/stage-21-points,tour-de-france,Points classification,None,
3,2021,gc,,1,,st4,race/tour-de-france/2021/gc,tour-de-france,General classification,None,
4,2021,etappe,18.07,72,,stage,race/tour-de-france/2021/stage-21,tour-de-france,Stage 21 - Chatou › Paris Champs-Élysées,None,108.4
...,...,...,...,...,...,...,...,...,...,...,...
75,2021,one_day,20.06,5,,stage,race/nc-slovenia/2021/result,nc-slovenia,National Championships Slovenia - Road Race (NC),[(NC)],172
76,2021,one_day,17.06,3,,chrono,race/nc-slovenia-itt/2021/result,nc-slovenia-itt,National Championships Slovenia - ITT (NC),[(NC)],31.5
78,2021,one_day,25.04,1,,stage,race/liege-bastogne-liege/2021/result,liege-bastogne-liege,Liège-Bastogne-Liège (1.UWT),[(1.UWT)],259.1
79,2021,one_day,21.04,DNS,,stage,race/la-fleche-wallone/2021/result,la-fleche-wallone,La Flèche Wallonne (1.UWT),[(1.UWT)],193.6


In [164]:
tadej[len(tadej_df['date']) > 5]

{'year': '2021',
 'type': 'gc',
 'date': '',
 'result': '1',
 'gc': '',
 'icon': 'st7',
 'race_ref': 'race/tour-de-france/2021/stage-21-kom',
 'race_name': 'tour-de-france',
 'race_detail': 'Mountains classification',
 'race_rank': '',
 'distance': ''}

In [67]:
base_url = 'https://www.procyclingstats.com/'

endpoint = 'rider/tadej-pogacar/'

year = '2021'

rider = 'POGAČAR Tadej'

url = base_url+endpoint+year

In [68]:
response = requests.get(url).content

soup = BeautifulSoup(response)

In [77]:
stage_races = soup.find_all('tr', {'data-main': '0'})

one_day_races = soup.find_all('tr', {'data-main': '1'})

In [88]:
master_ls = []
for o in one_day_races:
    dict = {}
    o = o.find_all('td')
    dict['type'] = 'one_day'
    dict['day'] = o[0].text
    dict['result'] = o[1].text
    dict['race_ref'] = o[4].a['href']
    dict['race_name'] = o[4].a.text
    dict['race_rank'] = o[4].find_all('span')[-1].text
    dict['distance'] = o[5].text
    master_ls.append(dict)

In [89]:
pd.DataFrame(master_ls)

,day,result,race_ref,race_name,race_rank,distance
0,09.10,1,race/il-lombardia/2021/result,Il Lombardia (1.UWT),(1.UWT),239
1,06.10,4,race/milano-torino/2021/result,Milano - Torino (1.Pro),51k,190
2,05.10,3,race/tre-valli-varesine/2021/result,Tre Valli Varesine (1.Pro),86k,196.7
3,02.10,DNF,race/giro-dell-emilia/2021/result,Giro dell'Emilia (1.Pro),(1.Pro),195.3
4,26.09,37,race/world-championship/2021/result,World Championships - Road Race (WC),(WC),268.3
5,19.09,10,race/world-championship-itt/2021/result,World Championships - ITT (WC),(WC),43.3
6,12.09,5,race/uec-road-european-championships/2021/result,European Continental Championships - Road Race...,68k,179.2
7,09.09,12,race/uec-road-european-championships-itt/2021/...,European Continental Championships - ITT (CC),(CC),22.4
8,29.08,DNF,race/bretagne-classic/2021/result,Bretagne Classic - Ouest-France (1.UWT),(1.UWT),251
9,24.07,3,race/olympic-games/2021/result,Olympic Games Road Race (Olympics),(Olympics),234


In [113]:
stage_races[0].find('span', class_='icon')['class'][-1]

'st6'

In [90]:
master_ls = []
for o in stage_races:
    dict = {}
    o = o.find_all('td')
    dict['type'] = 'stage_race'
    dict['day'] = o[0].text
    dict['result'] = o[1].text
    dict['race_ref'] = o[4].a['href']
    dict['race_name'] = o[4].a.text
    dict['race_rank'] = o[4].find_all('span')[-1].text
    dict['distance'] = o[5].text
    master_ls.append(dict)

In [91]:
pd.DataFrame(master_ls)

,day,result,race_ref,race_name,race_rank,distance
0,,1,race/tour-de-france/2021/stage-21-youth,Youth classification,,
1,,1,race/tour-de-france/2021/stage-21-kom,Mountains classification,,
2,,8,race/tour-de-france/2021/stage-21-points,Points classification,,
3,,1,race/tour-de-france/2021/gc,General classification,,
4,18.07,72,race/tour-de-france/2021/stage-21,Stage 21 - Chatou › Paris Champs-Élysées,,108.4
...,...,...,...,...,...,...
59,25.02,2,race/uae-tour/2021/stage-5,Stage 5 - Fujairah Marine Club › Jebel Jais,,170
60,24.02,20,race/uae-tour/2021/stage-4,Stage 4 - Al Marjan Island › Al Marjan Island,,204
61,23.02,1,race/uae-tour/2021/stage-3,Stage 3 - Strata Manufactoring › Jebel Hafeet,,166
62,22.02,4,race/uae-tour/2021/stage-2,Stage 2 (ITT) - Al Hudayriat Island › Al Huday...,,13
